In [8]:
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from copy import deepcopy
from tqdm import tqdm
from balu3.fs.sel    import sfs, clean
from sklearn.linear_model import LogisticRegression
from tensorflow.keras import layers
from keras.callbacks import ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from random import randint
from tensorflow.keras.layers import Dropout

import tensorflow as tf
import pandas as pd
import numpy as np
import torch
import keras

In [9]:
# Data historica

dfHistorico1 = pd.read_csv("dataHistórica/resultados_pasados_premier.csv")
dfHistorico1 = dfHistorico1.drop("Unnamed: 0",axis = 1)
dfHistorico1 = dfHistorico1.drop(["Position_last_year_home", "Position_last_year_away",
                                  "Points_last_year_home", "Points_last_year_away"],axis = 1) # Para hacer el merge con el segundo.

dfHistorico2 = pd.read_csv("dataHistórica/restoDeLigas.csv")
dfHistorico2 = dfHistorico2.drop("Unnamed: 0",axis = 1)

dfHistorico2['Date'] = pd.to_datetime(dfHistorico2['Date'], dayfirst=True, errors='coerce')
dfHistorico2['Date'] = dfHistorico2['Date'].dt.strftime('%Y-%m-%d')
dfHistorico2 = dfHistorico2[dfHistorico2["Date"] >"2010-01-01"] # Seleccion solo de partidos de 2010 en adelante (porque el otro dataframe tiene info desde ahí)


C:\Users\gfuen\AppData\Local\Temp\ipykernel_10196\788853108.py:11: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dfHistorico2['Date'] = pd.to_datetime(dfHistorico2['Date'], dayfirst=True, errors='coerce')


In [10]:
dfHistorico = pd.concat([dfHistorico1, dfHistorico2], axis=0)
dfHistorico = dfHistorico.reset_index(drop=True)

In [11]:
# Data tipo evento

dfEvent = pd.read_csv("dataWhoScored/WhoScoredTeamPerMatchSpatial1x1TimeDiv90.csv")
dfEvent = dfEvent.drop("Unnamed: 0",axis = 1)

In [12]:
# Ajuste manual de equipos:
# Los que no tengan match son un string vacio.

histToEventNameV1 = {
   "Arsenal": "Arsenal", "Aston Villa": "Aston Villa", "Barnsley": "", "Birmingham City": "Birmingham", "Blackburn": "Blackburn","Blackpool": "Blackpool",
   "Bolton" : "Bolton", "Bournemouth": "Bournemouth", "Bradford City": "", "Brentford": "Brentford", "Brighton": "Brighton", "Burnley": "Burnley", "Cardiff City": "Cardiff",
   "Charlton Ath": "", "Chelsea": "Chelsea", "Coventry City": "", "Crystal Palace": "Crystal Palace", "Derby County":"", "Everton": "Everton", "Fulham": "Fulham",
   "Huddersfield": "Huddersfield", "Hull City": "Hull", "Ipswich Town": "", "Leeds United": "Leeds", "Leicester City": "Leicester", "Liverpool": "Liverpool",
   "Manchester City": "Man City", "Manchester Utd": "Man Utd", "Middlesbrough": "Middlesbrough", "Newcastle Utd": "Newcastle", "Norwich City": "Norwich",
   "Nott'ham Forest": "Nottingham Forest", "Oldham Athletic": "", "Portsmouth": "", "QPR": "QPR", "Reading": "Reading", "Sheffield Utd": "Sheff Utd", "Sheffield Weds": "",
   "Southampton": "Southampton", "Stoke City": "Stoke", "Sunderland": "Sunderland", "Swansea City": "Swansea", "Swindon Town":  "", "Tottenham": "Tottenham", "Watford":"Watford",
   "West Brom": "WBA", "West Ham": "West Ham", "Wigan Athletic": "Wigan", "Wimbledon": "", "Wolves": "Wolves"
}
len(histToEventNameV1)

50

In [13]:
histToEventName = {
   "Arsenal": "Arsenal", "Aston Villa": "Aston Villa", "Barnsley": "", "Birmingham City": "Birmingham", "Blackburn": "Blackburn","Blackpool": "Blackpool",
   "Bolton" : "Bolton", "Bournemouth": "Bournemouth", "Bradford City": "", "Brentford": "Brentford", "Brighton": "Brighton", "Burnley": "Burnley", "Cardiff City": "Cardiff",
   "Charlton Ath": "", "Chelsea": "Chelsea", "Coventry City": "", "Crystal Palace": "Crystal Palace", "Derby County":"", "Everton": "Everton", "Fulham": "Fulham",
   "Huddersfield": "Huddersfield", "Hull City": "Hull", "Ipswich Town": "", "Leeds United": "Leeds", "Leicester City": "Leicester", "Liverpool": "Liverpool",
   "Manchester City": "Man City", "Manchester Utd": "Man Utd", "Middlesbrough": "Middlesbrough", "Newcastle Utd": "Newcastle", "Norwich City": "Norwich",
   "Nott'ham Forest": "Nottingham Forest", "Oldham Athletic": "", "Portsmouth": "", "QPR": "QPR", "Reading": "Reading", "Sheffield Utd": "Sheff Utd", "Sheffield Weds": "",
   "Southampton": "Southampton", "Stoke City": "Stoke", "Sunderland": "Sunderland", "Swansea City": "Swansea", "Swindon Town":  "", "Tottenham": "Tottenham", "Watford":"Watford",
   "West Brom": "WBA", "West Ham": "West Ham", "Wigan Athletic": "Wigan", "Wimbledon": "", "Wolves": "Wolves", #V1
   "Aachen":"","Ajaccio":"AC Ajaccio", "Alaves":"Deportivo Alaves", "Almeria": "Almeria", "Amiens":"Amiens","Angers":"Angers", "Arles":"Arles-Avignon", "Ascoli":"",
   "Atalanta":"Atalanta", "Ath Bilbao":"Athletic Club", "Ath Madrid":"Atletico", "Augsburg":"Augsburg", "Auxerre":"Auxerre", "Barcelona":"Barcelona", "Bari":"Bari",
   "Bastia":"SC Bastia", "Bayern Munich":"Bayern", "Benevento":"Benevento", "Betis":"Real Betis", "Bielefeld":"Arminia Bielefeld", "Bochum": "Bochum", "Bologna":"Bologna",
   "Bordeaux":"Bordeaux", "Boulogne": "", "Braunschweig": "Eintracht Braunschweig", "Brescia":"Brescia", "Brest":"Brest", "Cadiz":"Cadiz", "Caen": "Caen", "Cagliari":"Cagliari",
   "Carpi":"AC Carpi", "Catania":"Catania", "Celta":"Celta Vigo", "Cesena": "Cesena", "Chievo":"Chievo", "Clermont":"Clermont Foot", "Cordoba": "Cordoba", "Cottbus":"",
   "Cremonese":"Cremonese", "Crotone":"Crotone", "Darmstadt":"Darmstadt", "Dijon": "Dijon", "Dortmund":"Borussia Dortmund", "Duisburg":"", "Eibar":"Eibar", "Ein Frankfurt":"Eintracht Frankfurt",
   "Elche": "Elche", "Empoli": "Empoli", "Espanol": "Espanyol", "Evian Thonon Gaillard": "Evian", "FC Koln": "FC Koln", "Fiorentina": "Fiorentina", "Fortuna Dusseldorf": "Fortuna Duesseldorf",
   "Freiburg":"Freiburg", "Frosinone": "Frosinone", "Genoa": "Genoa", "Getafe": "Getafe", "Gimnastic":"", "Girona": "Girona", "Granada": "Granada", "Grenoble":"",
   "Greuther Furth": "Greuther Fuerth", "Guingamp": "Guingamp", "Hamburg":"Hamburg", "Hannover":"Hannover", "Hansa Rostock":"", "Heidenheim":"FC Heidenheim", "Hercules":"Hercules",
   "Hertha":"Hertha Berlin", "Hoffenheim":"Hoffenheim", "Huesca": "SD Huesca", "Ingolstadt": "Ingolstadt", "Inter": "Inter", "Juventus":"Juventus", "Kaiserslautern":"Kaiserslautern",
   "Karlsruhe":"", "La Coruna": "Deportivo", "Las Palmas": "Las Palmas", "Lazio": "Lazio", "Le Havre": "Le Havre", "Le Mans":"", "Lecce": "Lecce", "Leganes":"Leganes", "Lens": "Lens",
   "Levante": "Levante", "Leverkusen": "Leverkusen", "Lille":"Lille", "Livorno": "Livorno", "Lorient":"Lorient", "Lyon": "Lyon", "M'gladbach": "Borussia M.Gladbach", "Mainz": "Mainz",
   "Malaga":"Malaga", "Mallorca":"Mallorca", "Messina": "", "Metz": "Metz", "Milan": "AC Milan", "Monaco": "Monaco", "Montpellier": "Montpellier", "Monza": "Monza", "Murcia":"",
   "Nancy":"Nancy", "Nantes":"Nantes", "Napoli":"Napoli", "Nice":"Nice", "Nimes":"Nimes", "Novara": "Novara", "Numancia":"", "Nurnberg":"Nuernberg", "Osasuna": "Osasuna",
   "Paderborn": "Paderborn", "Palermo": "Palermo", "Paris SG": "PSG", "Parma": "Parma Calcio 1913", "Pescara":"Pescara", "RB Leipzig": "RBL","Real Madrid":"Real Madrid", 
   "Recreativo":"", "Reggina":"", "Reims": "Reims", "Rennes":"Rennes", "Roma":"Roma", "Salernitana": "Salernitana", "Sampdoria":"Sampdoria", "Santander": "Racing Santander",
   "Sassuolo":"Sassuolo", "Schalke 04": "Schalke", "Sedan":"", "Sevilla":"Sevilla", "Siena": "Siena", "Sochaux": "Sochaux", "Sociedad": "Real Sociedad", "Sp Gijon":"Sporting Gijon",
   "Spal":"SPAL 2013", "Spezia": "Spezia", "St Etienne":"Saint-Etienne", "St Pauli":"St. Pauli", "Strasbourg":"Strasbourg", "Stuttgart": "Stuttgart", "Tenerife":"", "Torino":"Torino",
   "Toulouse":"Toulouse", "Treviso":"", "Troyes":"Troyes", "Udinese": "Udinese", "Union Berlin":"Union Berlin", "Valencia": "Valencia", "Valenciennes": "Valenciennes", "Valladolid": "Real Valladolid",
   "Vallecano": "Rayo Vallecano", "Venezia":"Venezia", "Verona":"Verona", "Villarreal": "Villarreal", "Werder Bremen": "Werder Bremen", "Wolfsburg": "Wolfsburg", "Xerez": "", "Zaragoza": "Real Zaragoza"
}
len(histToEventName)

207

In [14]:
# Para buscar nombres

a = dfHistorico["Team_home"].unique().tolist()
a.sort()
print(a)

['Ajaccio', 'Ajaccio GFCO', 'Alaves', 'Almeria', 'Amiens', 'Angers', 'Arles', 'Arsenal', 'Aston Villa', 'Atalanta', 'Ath Bilbao', 'Ath Madrid', 'Augsburg', 'Auxerre', 'Barcelona', 'Bari', 'Barnsley', 'Bastia', 'Bayern Munich', 'Benevento', 'Betis', 'Bielefeld', 'Birmingham City', 'Blackburn', 'Blackpool', 'Bochum', 'Bologna', 'Bolton', 'Bordeaux', 'Boulogne', 'Bournemouth', 'Bradford City', 'Braunschweig', 'Brentford', 'Brescia', 'Brest', 'Brighton', 'Burnley', 'Cadiz', 'Caen', 'Cagliari', 'Cardiff City', 'Carpi', 'Catania', 'Celta', 'Cesena', 'Charlton Ath', 'Chelsea', 'Chievo', 'Clermont', 'Cordoba', 'Coventry City', 'Cremonese', 'Crotone', 'Crystal Palace', 'Darmstadt', 'Derby County', 'Dijon', 'Dortmund', 'Eibar', 'Ein Frankfurt', 'Elche', 'Empoli', 'Espanol', 'Everton', 'Evian Thonon Gaillard', 'FC Koln', 'Fiorentina', 'Fortuna Dusseldorf', 'Freiburg', 'Frosinone', 'Fulham', 'Genoa', 'Getafe', 'Girona', 'Granada', 'Grenoble', 'Greuther Furth', 'Guingamp', 'Hamburg', 'Hannover', 'H

In [15]:
# Para buscar nombres

a = dfEvent["Team"].unique().tolist()
a.sort()
print(a)

['AC Ajaccio', 'AC Carpi', 'AC Milan', 'Almeria', 'Amiens', 'Angers', 'Arles-Avignon', 'Arminia Bielefeld', 'Arsenal', 'Aston Villa', 'Atalanta', 'Athletic Club', 'Atletico', 'Augsburg', 'Auxerre', 'Barcelona', 'Bari', 'Bayern', 'Benevento', 'Birmingham', 'Blackburn', 'Blackpool', 'Bochum', 'Bologna', 'Bolton', 'Bordeaux', 'Borussia Dortmund', 'Borussia M.Gladbach', 'Bournemouth', 'Brentford', 'Brescia', 'Brest', 'Brighton', 'Burnley', 'Cadiz', 'Caen', 'Cagliari', 'Cardiff', 'Catania', 'Celta Vigo', 'Cesena', 'Chelsea', 'Chievo', 'Clermont Foot', 'Cordoba', 'Cremonese', 'Crotone', 'Crystal Palace', 'Darmstadt', 'Deportivo', 'Deportivo Alaves', 'Dijon', 'Eibar', 'Eintracht Braunschweig', 'Eintracht Frankfurt', 'Elche', 'Empoli', 'Espanyol', 'Everton', 'Evian', 'FC Heidenheim', 'FC Koln', 'Fiorentina', 'Fortuna Duesseldorf', 'Freiburg', 'Frosinone', 'Fulham', 'GFC Ajaccio', 'Genoa', 'Getafe', 'Girona', 'Granada', 'Greuther Fuerth', 'Guingamp', 'Hamburg', 'Hannover', 'Hercules', 'Hertha B

In [16]:
# Nombres de data histórica a nombres de tipo evento.

dfHistorico["Team_home"] = dfHistorico["Team_home"].map(histToEventName)
dfHistorico["Team_away"] = dfHistorico["Team_away"].map(histToEventName)

In [17]:
# Comprobamos que no esten datos vacios

dfHistorico.isnull().sum() 

Wk                                 0
Date                               0
Team_home                        284
Team_away                        281
Goals_home                         0
Goals_away                         0
score_prom_home                    0
score_prom_away                    0
wins_home                          0
wins_away                          0
ties_home                          0
ties_away                          0
losses_home                        0
losses_away                        0
total_goals_home                   0
total_goals_away                   0
total_goals_conceded_home          0
total_goals_conceded_away          0
points_home                        0
points_away                        0
streak_home                        0
streak_away                        0
wins_in_last_5_matches_home        0
wins_in_last_5_matches_away        0
ties_in_last_5_matches_home        0
ties_in_last_5_matches_away        0
losses_in_last_5_matches_home      0
l

In [18]:
dfHistorico

,Wk,Date,Team_home,Team_away,Goals_home,Goals_away,score_prom_home,score_prom_away,wins_home,wins_away,...,points_away,streak_home,streak_away,wins_in_last_5_matches_home,wins_in_last_5_matches_away,ties_in_last_5_matches_home,ties_in_last_5_matches_away,losses_in_last_5_matches_home,losses_in_last_5_matches_away,result
0,1.0,1993-08-14,Liverpool,,2.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,1.0,1993-08-14,Chelsea,Blackburn,1.0,2.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
2,1.0,1993-08-14,Aston Villa,QPR,4.0,1.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,1.0,1993-08-14,,,0.0,3.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
4,1.0,1993-08-14,Arsenal,,0.0,3.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31974,23.0,2024-02-24,Union Berlin,FC Heidenheim,2.0,2.0,0.954545,1.409091,7.0,7.0,...,27.0,2.0,0.0,3.0,1.0,1.0,3.0,1.0,1.0,0
31975,23.0,2024-02-24,Werder Bremen,Darmstadt,1.0,1.0,1.409091,1.045455,8.0,2.0,...,12.0,1.0,0.0,4.0,0.0,0.0,2.0,1.0,3.0,0
31976,23.0,2024-02-24,Bayern,RBL,2.0,1.0,2.772727,2.181818,16.0,12.0,...,40.0,0.0,3.0,2.0,2.0,0.0,1.0,3.0,2.0,1
31977,23.0,2024-02-25,Eintracht Frankfurt,Wolfsburg,2.0,2.0,1.545455,1.181818,8.0,6.0,...,24.0,2.0,1.0,1.0,0.0,3.0,4.0,1.0,1.0,0


In [19]:
# Primero revisamos si estan todos los partidos compatibles, para que sea más robusta la solución

test = []
for i in tqdm(range(len(dfHistorico))):
    stringHome = dfHistorico.iloc[i].Team_home
    stringAway = dfHistorico.iloc[i].Team_away
    date = dfHistorico.iloc[i].Date

    dfAux = dfEvent[dfEvent.Date == date]

    dfHome = dfAux[dfAux.Team == stringHome].add_prefix("Home")
    dfAway = dfAux[dfAux.Team == stringAway].add_prefix("Away")

    # Dropear valores redundantes
    dfHome = dfHome.drop(["HomeTeam","HomeDate","HomeGoals","Homegame_id"], axis = 1)
    dfAway = dfAway.drop(["AwayTeam","AwayDate","AwayGoals","Awaygame_id"], axis = 1)


    dfHome.reset_index(drop=True, inplace=True)
    dfAway.reset_index(drop=True, inplace=True)

    concat = pd.concat([dfHome, dfAway], axis = 1)
    test.append(concat)
    

  0%|          | 0/31979 [00:00<?, ?it/s]

100%|██████████| 31979/31979 [01:30<00:00, 355.09it/s]


In [20]:
# Buscamos los partidos que tenemos información

indexHistorical = []

for i in range(len(test)):
    if len(test[i]) == 1:
        indexHistorical.append(i)

In [21]:
# Creamos el dataframe de partidos a buscar

dfHistoricoCompatible = dfHistorico.iloc[indexHistorical].reset_index(drop=True)
dfHistoricoCompatible

,Wk,Date,Team_home,Team_away,Goals_home,Goals_away,score_prom_home,score_prom_away,wins_home,wins_away,...,points_away,streak_home,streak_away,wins_in_last_5_matches_home,wins_in_last_5_matches_away,ties_in_last_5_matches_home,ties_in_last_5_matches_away,losses_in_last_5_matches_home,losses_in_last_5_matches_away,result
0,1.0,2010-08-14,Bolton,Fulham,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,1.0,2010-08-14,Wigan,Blackpool,0.0,4.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
2,1.0,2010-08-14,Tottenham,Man City,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,1.0,2010-08-14,Aston Villa,West Ham,3.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,1.0,2010-08-14,Chelsea,WBA,6.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22706,23.0,2024-02-23,Leverkusen,Mainz,2.0,1.0,2.590909,0.772727,18.0,2.0,...,15.0,22.0,1.0,4.0,1.0,1.0,1.0,0.0,3.0,1
22707,23.0,2024-02-24,Borussia M.Gladbach,Bochum,5.0,2.0,1.636364,1.181818,5.0,5.0,...,25.0,0.0,3.0,0.0,1.0,2.0,3.0,3.0,1.0,1
22708,23.0,2024-02-24,Stuttgart,FC Koln,1.0,1.0,2.318182,0.681818,15.0,3.0,...,16.0,4.0,0.0,4.0,1.0,0.0,2.0,1.0,2.0,0
22709,23.0,2024-02-24,Union Berlin,FC Heidenheim,2.0,2.0,0.954545,1.409091,7.0,7.0,...,27.0,2.0,0.0,3.0,1.0,1.0,3.0,1.0,1.0,0


In [22]:
def featureMod(matchData):

    # Solución 1 promedio:
    return pd.DataFrame(matchData.mean())

goals_succ = dfEvent.filter(regex='Goal_Successful_bin').columns.tolist()
goals_unsucc = dfEvent.filter(regex='Goal_Unsuccessful_bin').columns.tolist()
goals_succ.extend(goals_unsucc)
goals = goals_succ

test = []
number_matches = 10

for i in tqdm(range(len(dfHistoricoCompatible))):
    stringHome = dfHistoricoCompatible.iloc[i].Team_home
    stringAway = dfHistoricoCompatible.iloc[i].Team_away
    date = dfHistoricoCompatible.iloc[i].Date

    dfAux = dfEvent[dfEvent.Date < date].drop(goals, axis=1)

    dfHome = dfAux[dfAux.Team == stringHome].iloc[-number_matches:].add_prefix("Home")
    dfAway = dfAux[dfAux.Team == stringAway].iloc[-number_matches:].add_prefix("Away")

    # Dropear valores redundantes

    dfHome = dfHome.drop(["HomeTeam","HomeDate","HomeGoals","Homegame_id"], axis = 1)
    dfAway = dfAway.drop(["AwayTeam","AwayDate","AwayGoals","Awaygame_id"], axis = 1)

    dfHome.reset_index(drop=True, inplace=True)
    dfAway.reset_index(drop=True, inplace=True)

    concat = pd.concat([dfHome, dfAway], axis = 1).reset_index(drop=True)

    if len(concat) == 0:
        pass
    else:
        mod = featureMod(concat)
        fila =  pd.concat([dfHistoricoCompatible.iloc[i:i+1].reset_index(drop=True), mod.T], axis=1)
        test.append(fila)

df = pd.concat(test, axis = 0)

100%|██████████| 22711/22711 [03:37<00:00, 104.56it/s]


In [24]:
# Revisamos si hay valores null
df.isnull().sum()

Wk                                                 0
Date                                               0
Team_home                                        250
Team_away                                        249
Goals_home                                         0
                                                ... 
AwayOffsideGiven_Unsuccessful_bin_0_time_0-90    292
AwayPunch_Successful_bin_0_time_0-90             292
AwayPunch_Unsuccessful_bin_0_time_0-90           292
AwayError_Successful_bin_0_time_0-90             292
AwayError_Unsuccessful_bin_0_time_0-90           292
Length: 129, dtype: int64

In [27]:
# Si hay, pero son poco, dropeemoslo

df = df.dropna()

In [28]:
# Eliminar las filas donde score_prom es 0, puesto que no hay info.

df = df[df["score_prom_home"] + df["score_prom_away"] != 0 ] # es un y 0 lógico

In [29]:
df.shape

(21548, 129)

# Crear DF balanceado

In [30]:
matches_train = df[df["Date"] < "2020-01-01"]
matches_test = df[df["Date"] >= "2020-01-01"]

print(sum(matches_train["result"]==0))
print(sum(matches_train["result"]==1))
print(sum(matches_train["result"]==2))


4125
7557
4717


In [31]:
n = 4125 # Manual
AUX = pd.DataFrame()
for i in range(2):

    aux = pd.concat([matches_train[matches_train["result"]==0].sample(n),     
                            matches_train[matches_train["result"]==1].sample(n), 
                            matches_train[matches_train["result"]==2].sample(n)], axis=0)
    AUX = pd.concat([AUX, aux], axis=0)
matches_train = AUX

In [32]:
#Testeamos las  proporciones
print(f"Proporciones: Train : {len(matches_train)/ (len(matches_train)+ len(matches_test))}, Test: {len(matches_test)/ (len(matches_train)+ len(matches_test))}")

# Dropeamos lo que no nos interesa
result_train = deepcopy(matches_train["result"])
result_test = deepcopy(matches_test["result"])

matches_train = matches_train.drop(columns = ["Date", "Team_home", "Team_away", "result", "Goals_home", "Goals_away", "Wk"])
matches_test = matches_test.drop(columns = ["Date", "Team_home", "Team_away", "result", "Goals_home", "Goals_away", "Wk"])

Proporciones: Train : 0.8277868825044316, Test: 0.17221311749556842


In [33]:
X = matches_train.to_numpy()
scaler = MinMaxScaler()
XBalance_train = scaler.fit_transform(X)
yBalance_train = deepcopy(result_train)

XBalance_test = scaler.transform(matches_test.to_numpy())
yBalance_test = deepcopy(result_test)

# Si queremos probar el clean descomentar lo siguiente

sclean  = clean(XBalance_train)      # indices of selected features
XBalance_train = deepcopy(XBalance_train[:,sclean])
XBalance_test = deepcopy(XBalance_test[:,sclean])

# Crear data full (mezclada)

In [34]:
matches_train = df[df["Date"] < "2020-01-01"]
matches_test = df[df["Date"] >= "2020-01-01"]

#Testeamos las  proporciones
print(f"Proporciones: Train : {len(matches_train)/ len(df)}, Test: {len(matches_test)/ len(df)}")

# Dropeamos lo que no nos interesa
result_train = deepcopy(matches_train["result"])
result_test = deepcopy(matches_test["result"])

matches_train = matches_train.drop(columns = ["Date", "Team_home", "Team_away", "result", "Goals_home", "Goals_away", "Wk"])
matches_test = matches_test.drop(columns = ["Date", "Team_home", "Team_away", "result", "Goals_home", "Goals_away", "Wk"])

Proporciones: Train : 0.7610451085947652, Test: 0.23895489140523482


In [35]:
X = matches_train.to_numpy()
scaler = MinMaxScaler()
XFull_train = scaler.fit_transform(X)
yFull_train = deepcopy(result_train)

XFull_test = scaler.transform(matches_test.to_numpy())
yFull_test = deepcopy(result_test)

# Si queremos probar el clean descomentar lo siguiente

sclean  = clean(XFull_train)      # indices of selected features
XFull_train = deepcopy(XFull_train[:,sclean])
XFull_test = deepcopy(XFull_test[:,sclean])

# Creacion de data solo eventual

In [36]:
matches_train = df[df["Date"] < "2020-01-01"]
matches_test = df[df["Date"] >= "2020-01-01"]

#Testeamos las  proporciones
print(f"Proporciones: Train : {len(matches_train)/ len(df)}, Test: {len(matches_test)/ len(df)}")

# Dropeamos lo que no nos interesa
result_train = deepcopy(matches_train["result"])
result_test = deepcopy(matches_test["result"])

columnasHistoricas = list(dfHistoricoCompatible.columns)

matches_train = matches_train.drop(columns = columnasHistoricas)
matches_test = matches_test.drop(columns = columnasHistoricas)

#### Para el proximo
columnasEventuales = list(matches_train.columns)

Proporciones: Train : 0.7610451085947652, Test: 0.23895489140523482


In [37]:
X = matches_train.to_numpy()
scaler = MinMaxScaler()
XEvent_train = scaler.fit_transform(X)
yEvent_train = deepcopy(result_train)

XEvent_test = scaler.transform(matches_test.to_numpy())
yEvent_test = deepcopy(result_test)

# Si queremos probar el clean descomentar lo siguiente

sclean  = clean(XEvent_train)      # indices of selected features
XEvent_train = deepcopy(XEvent_train[:,sclean])
XEvent_test = deepcopy(XEvent_test[:,sclean])

# Crear data solo histórica

In [38]:
matches_train = df[df["Date"] < "2020-01-01"]
matches_test = df[df["Date"] >= "2020-01-01"]

#Testeamos las  proporciones
print(f"Proporciones: Train : {len(matches_train)/ len(df)}, Test: {len(matches_test)/ len(df)}")

# Dropeamos lo que no nos interesa
result_train = deepcopy(matches_train["result"])
result_test = deepcopy(matches_test["result"])

matches_train = matches_train.drop(columns = columnasEventuales)
matches_test = matches_test.drop(columns = columnasEventuales)

matches_train = matches_train.drop(columns = ["Date", "Team_home", "Team_away", "result", "Goals_home", "Goals_away", "Wk"])
matches_test = matches_test.drop(columns = ["Date", "Team_home", "Team_away", "result", "Goals_home", "Goals_away", "Wk"])

Proporciones: Train : 0.7610451085947652, Test: 0.23895489140523482


In [39]:
X = matches_train.to_numpy()
scaler = MinMaxScaler()
XHist_train = scaler.fit_transform(X)
yHist_train = deepcopy(result_train)

XHist_test = scaler.transform(matches_test.to_numpy())
yHist_test = deepcopy(result_test)

# Si queremos probar el clean descomentar lo siguiente

sclean  = clean(XHist_train)      # indices of selected features
XHist_train = deepcopy(XHist_train[:,sclean])
XHist_test = deepcopy(XHist_test[:,sclean])

# Modelo 1, TableNet 

## Modelo solo histórico

In [56]:
clf = TabNetClassifier(
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=1e-3),
    scheduler_params={"step_size":50,
                        "gamma":0.9},
    scheduler_fn=torch.optim.lr_scheduler.StepLR,
    mask_type='entmax', # "sparsemax" "entmax"
)
    
clf.fit(XHist_train, yHist_train,
    eval_set=[(XHist_train, yHist_train), (XHist_test, yHist_test)], 
    eval_name=['train', 'valid'], 
    eval_metric=['accuracy'], 
    max_epochs=1000 , patience=50, 
    batch_size=28, drop_last=False)


predHist = clf.predict(XHist_test)
predHistTrain = clf.predict(XHist_train)

epoch 0  | loss: 1.15052 | train_accuracy: 0.45911 | valid_accuracy: 0.43931 |  0:00:19s
epoch 1  | loss: 1.06732 | train_accuracy: 0.47625 | valid_accuracy: 0.46145 |  0:00:39s
epoch 2  | loss: 1.04782 | train_accuracy: 0.49473 | valid_accuracy: 0.47271 |  0:01:00s
epoch 3  | loss: 1.0358  | train_accuracy: 0.49753 | valid_accuracy: 0.48398 |  0:01:19s
epoch 4  | loss: 1.02935 | train_accuracy: 0.50046 | valid_accuracy: 0.48068 |  0:01:38s
epoch 5  | loss: 1.02346 | train_accuracy: 0.50192 | valid_accuracy: 0.4898  |  0:01:57s
epoch 6  | loss: 1.02315 | train_accuracy: 0.50082 | valid_accuracy: 0.48398 |  0:02:16s
epoch 7  | loss: 1.01989 | train_accuracy: 0.50479 | valid_accuracy: 0.48611 |  0:02:34s
epoch 8  | loss: 1.01624 | train_accuracy: 0.50552 | valid_accuracy: 0.48708 |  0:02:53s
epoch 9  | loss: 1.01477 | train_accuracy: 0.50649 | valid_accuracy: 0.48864 |  0:03:12s
epoch 10 | loss: 1.01538 | train_accuracy: 0.50814 | valid_accuracy: 0.48534 |  0:03:31s
epoch 11 | loss: 1.01

KeyboardInterrupt: 

## Modelo solo eventual

In [ ]:
clf = TabNetClassifier(
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=1e-3),
    scheduler_params={"step_size":50,
                        "gamma":0.9},
    scheduler_fn=torch.optim.lr_scheduler.StepLR,
    mask_type='entmax', # "sparsemax" "entmax"
    )
    
clf.fit(XEvent_train, yEvent_train,
    eval_set=[(XEvent_train, yEvent_train), (XEvent_test, yEvent_test)], 
    eval_name=['train', 'valid'], 
    eval_metric=['accuracy'], 
    max_epochs=1000 , patience=50, 
    batch_size=28, drop_last=False)

predEvent = clf.predict(XEvent_test)
predEventTrain = clf.predict(XEvent_train)

## Modelo completo

In [ ]:
clf = TabNetClassifier(
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=1e-3),
    scheduler_params={"step_size":50,
                        "gamma":0.99},
    scheduler_fn=torch.optim.lr_scheduler.StepLR,
    mask_type='entmax', # "sparsemax" "entmax"
    )
    
clf.fit(XFull_train, yFull_train,
    eval_set=[(XFull_train, yFull_train), (XFull_test, yFull_test)], 
    eval_name=['train', 'valid'], 
    eval_metric=['accuracy'], 
    max_epochs=1000 , patience=50, 
    batch_size=16, drop_last=False)

predFull = clf.predict(XFull_test)
predFullTrain = clf.predict(XFull_train)

## Modelo completo Balanceado

In [ ]:
clf = TabNetClassifier(
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=1e-3),
    scheduler_params={"step_size":50,
                        "gamma":0.9},
    scheduler_fn=torch.optim.lr_scheduler.StepLR,
    mask_type='entmax', # "sparsemax" "entmax"
    )
    
clf.fit(XBalance_train, yBalance_train,
    eval_set=[(XBalance_train, yBalance_train), (XBalance_test, yBalance_test)], 
    eval_name=['train', 'valid'], 
    eval_metric=['accuracy'], 
    max_epochs=1000 , patience=50, 
    batch_size=28, drop_last=False)

predBalance = clf.predict(XBalance_test)
predBalanceTrain = clf.predict(XBalance_train)

## Evaluaciones

In [ ]:
# 0 = empate, 1 = home, 2, away

In [ ]:
# Modelo histórico

import matplotlib.pyplot as plt
import seaborn as sns

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(yHist_test, predHist)

# Etiquetas de clase
class_labels = ["Empate", "Home Win", "Away win"]

# Crear figura
plt.figure(figsize=(8, 6))
sns.set(font_scale=1.2) # Ajustar el tamaño de la fuente para una mejor legibilidad
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap="Blues", xticklabels=class_labels, yticklabels=class_labels)

# Añadir etiquetas
plt.xlabel('Predicción')
plt.ylabel('Etiqueta Real')
plt.title('Matriz de Confusión')

# Mostrar la matriz de confusión
plt.show()

In [ ]:
# Modelo eventual

import matplotlib.pyplot as plt
import seaborn as sns

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(list(yEvent_test), predEvent)

# Etiquetas de clase
class_labels = ["Empate", "Home Win", "Away win"]

# Crear figura
plt.figure(figsize=(8, 6))
sns.set(font_scale=1.2) # Ajustar el tamaño de la fuente para una mejor legibilidad
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap="Blues", xticklabels=class_labels, yticklabels=class_labels)

# Añadir etiquetas
plt.xlabel('Predicción')
plt.ylabel('Etiqueta Real')
plt.title('Matriz de Confusión')

# Mostrar la matriz de confusión
plt.show()

In [ ]:
# Modelo completo 

import matplotlib.pyplot as plt
import seaborn as sns

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(yFull_test, predFull)

# Etiquetas de clase
class_labels = ["Empate", "Home Win", "Away win"]

# Crear figura
plt.figure(figsize=(8, 6))
sns.set(font_scale=1.2) # Ajustar el tamaño de la fuente para una mejor legibilidad
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap="Blues", xticklabels=class_labels, yticklabels=class_labels)

# Añadir etiquetas
plt.xlabel('Predicción')
plt.ylabel('Etiqueta Real')
plt.title('Matriz de Confusión')

# Mostrar la matriz de confusión
plt.show()

In [ ]:
print(classification_report(yHist_test, predFull))

In [ ]:
# Modelo Balanceado

import matplotlib.pyplot as plt
import seaborn as sns

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(yBalance_test, predBalance)

# Etiquetas de clase
class_labels = ["Empate", "Home Win", "Away win"]

# Crear figura
plt.figure(figsize=(8, 6))
sns.set(font_scale=1.2) # Ajustar el tamaño de la fuente para una mejor legibilidad
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap="Blues", xticklabels=class_labels, yticklabels=class_labels)

# Añadir etiquetas
plt.xlabel('Predicción')
plt.ylabel('Etiqueta Real')
plt.title('Matriz de Confusión')

# Mostrar la matriz de confusión
plt.show()

In [ ]:
print(classification_report(yBalance_test, predBalance))

### Modelo 2 Regresion logistica

In [57]:
# Crear y entrenar el modelo de regresión logística
model = LogisticRegression()
model.fit(XFull_train, yFull_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(XFull_test)

# Calcular la precisión del modelo
accuracy = np.mean(y_pred == yFull_test)
print("Precisión del modelo:", accuracy)

Precisión del modelo: 0.5206836278889104


### Modelo 3 MLP

In [58]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [60]:
batch_size = 32
epoch = 100

# Define el modelo
# Construir el modelo
# Crear el modelo
model = tf.keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(XHist_train.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(3, activation='softmax', use_bias= True)
])
# Compilar el modelo
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  
              metrics=['accuracy'])

model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath="models\Hist",
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

history = model.fit(XHist_train, yHist_train, verbose=1, epochs=epoch, batch_size=batch_size,
                    validation_data=(XHist_test, yHist_test), callbacks=[model_checkpoint_callback])

Epoch 1/100
513/513 [==============================] - 2s 4ms/step - loss: 1.0172 - accuracy: 0.4963 - val_loss: 1.0111 - val_accuracy: 0.5013
Epoch 2/100
513/513 [==============================] - 2s 3ms/step - loss: 1.0034 - accuracy: 0.5085 - val_loss: 1.0123 - val_accuracy: 0.5020
Epoch 3/100
513/513 [==============================] - 2s 3ms/step - loss: 1.0012 - accuracy: 0.5117 - val_loss: 1.0124 - val_accuracy: 0.5042
Epoch 4/100
513/513 [==============================] - 1s 3ms/step - loss: 0.9994 - accuracy: 0.5131 - val_loss: 1.0189 - val_accuracy: 0.4964
Epoch 5/100
513/513 [==============================] - 1s 3ms/step - loss: 0.9986 - accuracy: 0.5124 - val_loss: 1.0100 - val_accuracy: 0.5038
Epoch 6/100
513/513 [==============================] - 2s 3ms/step - loss: 0.9979 - accuracy: 0.5130 - val_loss: 1.0093 - val_accuracy: 0.5050
Epoch 7/100
513/513 [==============================] - 1s 3ms/step - loss: 0.9972 - accuracy: 0.5147 - val_loss: 1.0124 - val_accuracy: 0.5007

KeyboardInterrupt: 

In [ ]:
batch_size = 32
epoch = 100

# Define el modelo
# Construir el modelo
# Crear el modelo
model = tf.keras.Sequential([
    layers.Dense(16, activation='relu', input_shape=(XEvent_train.shape[1],)),
    layers.Dense(16, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(3, activation='softmax', use_bias= True)
])
# Compilar el modelo
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  
              metrics=['accuracy'])

model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath="models\Event",
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

history = model.fit(XEvent_train, yEvent_train, verbose=1, epochs=epoch, batch_size=batch_size,
                    validation_data=(XEvent_test, yEvent_test), callbacks=[model_checkpoint_callback])

In [61]:
batch_size = 32
epoch = 100

# Define el modelo
# Construir el modelo
# Crear el modelo
model = tf.keras.Sequential([
    layers.Dense(256, activation='relu', input_shape=(XFull_train.shape[1],)),
    layers.Dense(256, activation='relu'),
    layers.Dense(256, activation='relu'),
    layers.Dense(3, activation='softmax', use_bias= True)
])
# Compilar el modelo
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  
              metrics=['accuracy'])

model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath="models\Full",
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

history = model.fit(XFull_train, yFull_train, verbose=1, epochs=epoch, batch_size=batch_size,
                    validation_data=(XFull_test, yFull_test), callbacks=[model_checkpoint_callback])

Epoch 1/100
513/513 [==============================] - 2s 4ms/step - loss: 1.0043 - accuracy: 0.5094 - val_loss: 0.9999 - val_accuracy: 0.5086
Epoch 2/100
513/513 [==============================] - 1s 3ms/step - loss: 0.9928 - accuracy: 0.5158 - val_loss: 1.0055 - val_accuracy: 0.5059
Epoch 3/100
513/513 [==============================] - 2s 3ms/step - loss: 0.9886 - accuracy: 0.5220 - val_loss: 0.9976 - val_accuracy: 0.5137
Epoch 4/100
513/513 [==============================] - 2s 3ms/step - loss: 0.9871 - accuracy: 0.5227 - val_loss: 0.9935 - val_accuracy: 0.5195
Epoch 5/100
513/513 [==============================] - 1s 2ms/step - loss: 0.9882 - accuracy: 0.5220 - val_loss: 0.9964 - val_accuracy: 0.5195
Epoch 6/100
513/513 [==============================] - 1s 3ms/step - loss: 0.9863 - accuracy: 0.5231 - val_loss: 0.9973 - val_accuracy: 0.5195
Epoch 7/100
513/513 [==============================] - 1s 3ms/step - loss: 0.9840 - accuracy: 0.5235 - val_loss: 0.9971 - val_accuracy: 0.5195

KeyboardInterrupt: 

### Modelo 4 LSTM

In [ ]:
from keras.utils import to_categorical

number_matches = 5
X = []
y = []
for i in tqdm(range(len(dfHistoricoCompatible))):
    stringHome = dfHistoricoCompatible.iloc[i].Team_home
    stringAway = dfHistoricoCompatible.iloc[i].Team_away
    date = dfHistoricoCompatible.iloc[i].Date

    dfAux = dfEvent[dfEvent.Date < date]

    dfHome = dfAux[dfAux.Team == stringHome].iloc[-number_matches:]
    dfAway = dfAux[dfAux.Team == stringAway].iloc[-number_matches:]
    
    # Generamos los resultados 
    # ids = dfAux[dfAux.Team == stringHome].iloc[-number_matches:].game_id

    # HistGoalsHome = dfAux[dfAux["game_id"].isin(ids) & (dfAux["Team"] == stringHome)]["Goals"].reset_index(drop=True)
    # HistRivalGoalsHome = dfAux[dfAux["game_id"].isin(ids) & (dfAux["Team"] != stringHome)]["Goals"].reset_index(drop=True)
    # HistResultsHome = list(np.sign(HistRivalGoalsHome - HistGoalsHome ) + 1)

    # ids = dfAux[dfAux.Team == stringAway].iloc[-number_matches:].game_id

    # HistGoalsAway = dfAux[dfAux["game_id"].isin(ids) & (dfAux["Team"] == stringAway)]["Goals"].reset_index(drop=True)
    # HistRivalGoalsAway = dfAux[dfAux["game_id"].isin(ids) & (dfAux["Team"] != stringAway)]["Goals"].reset_index(drop=True)
    # HistResultsAway = list(np.sign(HistRivalGoalsAway - HistGoalsAway ) + 1)
    # HistResultsHome.extend(HistResultsAway)

    # Dropear valores redundantes
    dfHome = dfHome.drop(["Team","Date","Goals","game_id"], axis = 1)
    dfAway = dfAway.drop(["Team","Date","Goals","game_id"], axis = 1)

    dfHome.reset_index(drop=True, inplace=True)
    dfAway.reset_index(drop=True, inplace=True)

    Xconcat = pd.concat([dfHome, dfAway], axis = 0).reset_index(drop=True)
    # yconcat = deepcopy(HistResultsHome)
    if len(Xconcat) == 2 * number_matches:
        X.append(Xconcat.values)
        # y.append(np.array(yconcat))
        y.append(dfHistoricoCompatible.iloc[i].result)
    else:
        pass

X = np.array(X)
y = to_categorical(np.array(y))


In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
# Dividir los datos en conjuntos de entrenamiento y prueba
split = int(0.8 * X.shape[0])
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam


# Definir el modelo LSTM
model = Sequential()
model.add(LSTM(50, return_sequences=False, input_shape=(10, 52)))
# model.add(LSTM(50))
model.add(Dense(3, activation='softmax'))

# Compilación del modelo
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Entrenamiento del modelo
model.fit(X_train, y_train, epochs=20, batch_size=4, validation_split=0.2)

# # Evaluación del modelo
loss, accuracy = model.evaluate(X_train, y_train)
print(f'Model Loss: {loss}')
print(f'Model Accuracy: {accuracy}')

In [ ]:
predLSTM = np.argmax(model.predict(X_test),axis=1)

In [ ]:
# Modelo completo 

import matplotlib.pyplot as plt
import seaborn as sns

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(np.argmax(y_test, axis=1), predLSTM)

# Etiquetas de clase
class_labels = ["Home Win", "Empate", "Away win"]

# Crear figura
plt.figure(figsize=(8, 6))
sns.set(font_scale=1.2) # Ajustar el tamaño de la fuente para una mejor legibilidad
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap="Blues", xticklabels=class_labels, yticklabels=class_labels)

# Añadir etiquetas
plt.xlabel('Predicción')
plt.ylabel('Etiqueta Real')
plt.title('Matriz de Confusión')

# Mostrar la matriz de confusión
plt.show()

In [ ]:
print(classification_report(np.argmax(y_test, axis=1), predLSTM))